# Intro to DTU

First, let's load the transcript abundance values computed with Salmon

We use `rtracklayer` this lib to load and handle GTF files
and `stringr` to make string operations easier 

In [1]:
library(rtracklayer)
library(stringr)

Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


At

Creates tx2gene, a object the holds the mapping between genes and transcripts

In [2]:
gtf.path <- '/biodb/genomes/homo_sapiens/GRCh38_96/GRCh38.96.gtf'
gtf <- rtracklayer::import(gtf.path)

In [3]:
head(gtf)

GRanges object with 6 ranges and 22 metadata columns:
      seqnames      ranges strand |   source       type     score     phase
         <Rle>   <IRanges>  <Rle> | <factor>   <factor> <numeric> <integer>
  [1]        1 11869-14409      + |   havana gene              NA      <NA>
  [2]        1 11869-14409      + |   havana transcript        NA      <NA>
  [3]        1 11869-12227      + |   havana exon              NA      <NA>
  [4]        1 12613-12721      + |   havana exon              NA      <NA>
  [5]        1 13221-14409      + |   havana exon              NA      <NA>
  [6]        1 12010-13670      + |   havana transcript        NA      <NA>
              gene_id gene_version   gene_name gene_source
          <character>  <character> <character> <character>
  [1] ENSG00000223972            5     DDX11L1      havana
  [2] ENSG00000223972            5     DDX11L1      havana
  [3] ENSG00000223972            5     DDX11L1      havana
  [4] ENSG00000223972            5     DDX1

Here we load the Salmon results using [`tximport`](https://bioconductor.org/packages/release/bioc/html/tximport.html)

In [4]:
tx2gene <- as.data.frame(gtf)[
    , c('gene_id', 'transcript_id', 'gene_name', 'transcript_name')]

In [5]:
library(tximport)

In [6]:
files <- Sys.glob('salmon/*/quant.sf')
txi <- tximport(
  files,
  type = "salmon",
  countsFromAbundance = "scaledTPM",
  txOut = TRUE
)

cts <- txi$counts

reading in files with read_tsv

1 
2 
3 
4 




If you know R, you can subset or filter transcript in the next step. **But** be careful, some operations may break the assumptions of the program you are using. Here we only change the conditions names, which we extract from the file names we use as input

In [7]:
head(txi$counts)

ENST00000631435.1,0,0,0,0
ENST00000434970.2,0,0,0,0
ENST00000448914.1,0,0,0,0
ENST00000415118.1,0,0,0,0
ENST00000604446.1,0,0,0,0
ENST00000603693.1,0,0,0,0


In [8]:
colnames(cts) <- str_split(files, '/', simplify = TRUE)[, 2]

In [9]:
rownames(cts) <- str_split(rownames(cts), '\\.', simplify = TRUE)[, 1]

In [10]:
head(cts)

,EGF_1,EGF_2,PBS_1,PBS_2
ENST00000631435,0,0,0,0
ENST00000434970,0,0,0,0
ENST00000448914,0,0,0,0
ENST00000415118,0,0,0,0
ENST00000604446,0,0,0,0
ENST00000603693,0,0,0,0


In [11]:
head(cts[rowSums(cts) > 1, ])

,EGF_1,EGF_2,PBS_1,PBS_2
ENST00000419783,63.314179,43.97932,62.001933,45.084436
ENST00000643797,0.000000,24.17524,6.911643,0.000000
ENST00000651740,22.432606,0.00000,0.000000,0.000000
ENST00000457194,1.273328,0.00000,0.000000,0.000000
ENST00000390289,0.000000,0.00000,10.902861,5.568154
ENST00000361390,1639.969021,1648.77401,1528.241587,1595.848738


For this analysis we use DRIMSeq, and the manual is [here](https://www.bioconductor.org/packages/release/bioc/vignettes/DRIMSeq/inst/doc/DRIMSeq.pdf)

In [12]:
matching <- intersect(rownames(cts), tx2gene$transcript_id)

In [13]:
gene_ids <- setNames(tx2gene$gene_id, tx2gene$transcript_id)

In [14]:
counts <- base::data.frame(
  gene_id = gene_ids[matching],
  feature_id = matching,
  cts[matching, ]
)

# Now we write the experimental design matrix

In [15]:
head(counts)

,gene_id,feature_id,EGF_1,EGF_2,PBS_1,PBS_2
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
ENST00000434970,ENSG00000237235,ENST00000434970,0,0,0,0
ENST00000448914,ENSG00000228985,ENST00000448914,0,0,0,0
ENST00000415118,ENSG00000223997,ENST00000415118,0,0,0,0
ENST00000604446,ENSG00000270824,ENST00000604446,0,0,0,0
ENST00000603693,ENSG00000270451,ENST00000603693,0,0,0,0
ENST00000603935,ENSG00000282089,ENST00000603935,0,0,0,0


In [16]:
samples <- base::data.frame(sample_id =  make.names(colnames(cts)))
samples$condition <- str_split(samples$sample_id, '_', simplify = TRUE)[, 1]

In [17]:
samples

sample_id,condition
<chr>,<chr>
EGF_1,EGF
EGF_2,EGF
PBS_1,PBS
PBS_2,PBS


Here we use DRIMSeq to model the usage of trascripts on the two conditions.
Next cell filter transcripts that have a low abundace:

In [29]:
library(DRIMSeq)
    
d <- dmDSdata(counts = counts, samples = samples)
d <- dmFilter(
  d,
  min_feature_expr = 30,
  min_feature_prop = 0.3,
  min_samps_gene_expr = 2,
  min_gene_expr = 50
)

In [34]:
# Number of genes
length(d@counts)

[1] 680

In [35]:
# Number of transcripts
sum(lengths(d@counts))

[1] 13888

In [36]:
design_full <- model.matrix(~condition, data = samples(d))

In [ ]:
d <- dmPrecision(d, design = design_full, prec_subset=0.001)

! Using a subset of 0.001 genes to estimate common precision !


! Using common_precision = 995.2027 as prec_init !




In [ ]:
plotData(d)

In [ ]:
d <- dmFit(d, design = design_full, verbose = 1)

In [ ]:
d <- dmTest(d, coef = "conditionEGF", verbose = 1)

In [ ]:
head(results(d))

In [ ]:
res <-  dplyr::filter(results(d), adj_pvalue < 0.05)

In [ ]:
head(res)

In [ ]:
res <-  dplyr::filter(results(d), pvalue < 0.05)

In [ ]:
head(dplyr::arrange(res, adj_pvalue))

In [ ]:
p <- plotProportions(
    d, plot_type = 'boxplot2', gene_id = 'ENSG00000160752', group_variable = "condition")

In [ ]:
plot(p)